# PyTorch Training and Serving in SageMaker "Script Mode"

Script mode is a training script format for PyTorch that lets you execute any PyTorch training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native PyTorch support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using PyTorch scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker PyTorch Serving container](https://github.com/aws/sagemaker-pytorch-serving-container). The PyTorch Serving container is the default inference method for script mode. For full documentation on deploying PyTorch models, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/doc/using_pytorch.rst#deploy-pytorch-models).

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

## Install SageMaker Python SDK

In [1]:
!pip install sagemaker --upgrade --ignore-installed --no-cache

    100% |████████████████████████████████| 296kB 13.3MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 50.7MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 100.1MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 97.1MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 95.0MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 57.5MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 96.6MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 63.1MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 95.3MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 64.5MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 98.5MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 104.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 87.4MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 94.4MB/s ta 0:00:01
    100% |█████████

In [15]:
!pip install torch==1.3.1 torchvision==0.4.2 --upgrade --ignore-installed --no-cache --user

    89% |████████████████████████████▋   | 656.8MB 80.2MB/s eta 0:00:01   11% |███▊                            | 85.2MB 79.6MB/s eta 0:00:09    13% |████▎                           | 98.2MB 81.6MB/s eta 0:00:08    27% |████████▊                       | 199.4MB 84.0MB/s eta 0:00:07███▋                  | 311.9MB 82.1MB/s eta 0:00:06   44% |██████████████▏                 | 324.3MB 86.0MB/s eta 0:00:05    | 421.2MB 85.0MB/s eta 0:00:04/s eta 0:00:04           | 430.5MB 79.4MB/s eta 0:00:04    59% |███████████████████             | 435.0MB 81.2MB/s eta 0:00:04    80% |█████████████████████████▉      | 592.9MB 76.2MB/s eta 0:00:02��███████████████████▋    | 633.6MB 74.8MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    97% |███████████████████████████████ | 714.0MB 78.5MB/s eta 0:00:01

    97% |███████████████████████████████▎| 717.7MB 80.1MB/s eta 0:00:01

    98% |█████████████�

    98% |███████████████████████████████▋| 725.7MB 78.9MB/s eta 0:00:01

    99% |███████████████████████████████▊| 729.3MB 80.7MB/s eta 0:00:01

    99% |████████████████████████████████| 733.2MB 65.5MB/s eta 0:00:01

    100% |████████████████████████████████| 734.6MB 97.6MB/s 


    100% |████████████████████████████████| 10.2MB 91.4MB/s ta 0:00:01███▋        | 7.5MB 86.1MB/s eta 0:00:01
    100% |████████████████████████████████| 20.2MB 95.5MB/s ta 0:00:01
    100% |████████████████████████████████| 2.1MB 12.1MB/s ta 0:00:01████▏                          | 337kB 75.0MB/s eta 0:00:01
docker-compose 1.24.1 has requirement requests!=2.11.0,!=2.12.2,!=2.18.0,<2.21,>=2.6.1, but you'll have requests 2.22.0 which is incompatible.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.13.49 which is incompatible.
awscli 1.16.283 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
  The scripts f2py, f2py3 and f2py3.6 are installed in '/home/ec2-user/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The scripts convert-caffe2-to-onnx and convert-onnx-to-caffe2 are installed in '/home/ec2-user/.local/bin' which is n

## Restart the Kernel to Recognize New Dependencies Above

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

In [ ]:
!pip3 list

## Create the SageMaker Session

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [ ]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

## Training Data

### Copy the Training Data to Your Notebook Disk

In [ ]:
local_data_path = './data'

In [1]:
from torchvision import datasets, transforms

normalization_mean = 0.1307
normalization_std = 0.3081

# download the dataset
# this will not only download data to ./mnist folder, but also load and transform (normalize) them
datasets.MNIST(local_data_path, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((normalization_mean,), (normalization_std,))
]))

ImportError: cannot import name 'PILLOW_VERSION'

In [ ]:
!ls -R {local_data_path}

### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [ ]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/pytorch-mnist/data'

In [ ]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print('Input spec (S3 path): {}'.format(training_data_uri))

In [ ]:
!aws s3 ls --recursive {training_data_uri}

## Train
### Training Script
The `pytorch_mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [2]:
!ls ./src/mnist_pytorch.py

./src/mnist_pytorch.py


You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [ ]:
!cat ./src/requirements.txt

### Train with SageMaker `PyTorch` Estimator

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure.  We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters.  In this case we are going to run our training job on two(2) `ml.p3.2xlarge` instances.  Alternatively, you can specify `ml.c4.xlarge` instances.  This example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)).  The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem of each worker, so our training script can simply read the data from disk.

### `fit` the Model (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

In [ ]:
from sagemaker.pytorch import PyTorch
import time

model_output_path = 's3://{}/sagemaker/pytorch-mnist/training-runs'.format(bucket)

mnist_estimator = PyTorch(
                  entry_point='mnist_pytorch.py',
                  source_dir='./src',
                  output_path=model_output_path,
                  role=role,
                  framework_version='1.3.1',
                  train_instance_count=1,
                  train_instance_type='ml.c5.2xlarge',
                  hyperparameters={
                    'epochs': 5,
                    'backend': 'gloo'
                  },
                  # Assuming the logline from the PyTorch training job is as follows:
                  #    Test set: Average loss: 0.3230, Accuracy: 9103/10000 (91%)
                  metric_definitions=[
                     {'Name':'test:loss', 'Regex':'Test set: Average loss: (.*?),'},
                     {'Name':'test:accuracy', 'Regex':'(.*?)%;'}
                  ]
)

mnist_estimator.fit(inputs={'training': training_data_uri},
                                        wait=False)

training_job_name = mnist_estimator.latest_training_job.name

print('training_job_name:  {}'.format(training_job_name))

Attach to a training job to monitor the logs.

_Note:  Each instance in the training job (2 in this example) will appear as a different color in the logs.  1 color per instance._

In [ ]:
mnist_estimator = PyTorch.attach(training_job_name=training_job_name)

## Option 1:  Perform Batch Predictions Directly in the Notebook

Use PyTorch Core to load the model from `model_output_path`

In [ ]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}/output/

In [ ]:
!aws --region {region} s3 cp {model_output_path}/{training_job_name}/output/model.tar.gz ./model/model.tar.gz

In [ ]:
!ls ./model

In [ ]:
!tar -xzvf ./model/model.tar.gz -C ./model

In [ ]:
# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
import torch

loaded_model = Net().to('cpu')
# single-machine multi-gpu case or single-machine or multi-machine cpu case
loaded_model = torch.nn.DataParallel(loaded_model)
print(loaded_model)

In [ ]:
loaded_model.load_state_dict(torch.load('./model/model.pth', map_location='cpu'))

In [ ]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=256, 
    shuffle=True
)

single_loaded_img = test_loader.dataset.data[0]
single_loaded_img = single_loaded_img.to('cpu')
single_loaded_img = single_loaded_img[None, None]
single_loaded_img = single_loaded_img.type('torch.FloatTensor') # instead of DoubleTensor

print(single_loaded_img.numpy())

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(single_loaded_img.numpy().reshape(28, 28), cmap='Greys')

In [ ]:
result = loaded_model(single_loaded_img)
prediction = result.max(1, keepdim=True)[1][0][0].numpy()
print(prediction)

## Option 2:  Create a SageMaker Endpoint and Perform REST-based Predictions

### Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `pytorch_mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c5.2xlarge')

### Invoke the Endpoint

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

The value of `data` is retrieved from the HTML above.

In [ ]:
print(data)

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### (Optional) Cleanup Endpoint

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)